In [2]:
import backtrader as bt
import pendulum
import yfinance as yf

In [3]:
start = pendulum.parse("2022-01-01")
end = pendulum.parse("2023-01-01")
df = yf.download(
    tickers="MSFT",
    # period="1y",
    interval="1h",
    start=start,
    end=end,
)

[*********************100%***********************]  1 of 1 completed


In [4]:
class SingleSMA(bt.SignalStrategy):
    def __init__(self, period=7):
        self.sma = bt.indicators.MovingAverageSimple(period=period)

    def next(self):
        if (
            self.sma[-1] < self.data.close
            # and self.sma[-1] < self.sma[-2]
            and not self.position
        ):
            self.buy()

        elif (
            self.sma[-1] > self.data.close
            # and self.sma[-1] > self.sma[-2]
            and self.position
        ):
            self.sell()

In [11]:
class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=7,  # period for the fast moving average
        pslow=26   # period for the slow moving average
    )

    def __init__(self, pfast, pslow):
        sma1 = bt.ind.SMA(period=pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.log(f"BUY at {self.data.close}")
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position

In [13]:
from IPython.display import clear_output


max_value = 0
opt_slow = 0
opt_fast = 0

for pfast in range(7, 13, 2):
    for pslow in range(24, 34, 4):
        cerebro = bt.Cerebro()
        cerebro.addwriter(bt.WriterFile)
        cerebro.addstrategy(SmaCross, pfast, pslow)

        data = bt.feeds.PandasData(
            dataname=df,
            timeframe=bt.TimeFrame.Minutes,
            compression=60,
        )

        cerebro.adddata(data)
        result = cerebro.run()

        value = cerebro.broker.get_value()
        if value > max_value:
            max_value = value
            opt_fast = pfast
            opt_slow = pslow
            
        clear_output()
        print(f"{pfast}-{pslow} -> {value}, max={max_value}")

AttributeError: 'Lines_LineSeries_LineIterator_DataAccessor_Strateg' object has no attribute 'log'

In [10]:
max_value

9959.075073242188

In [ ]:
cerebro.plot(
    iplot=False,
    #  Format string for the display of ticks on the x axis
    fmt_x_ticks="%Y-%b-%d %H:%M",
    # Format string for the display of data points values
    fmt_x_data="%Y-%b-%d %H:%M",
)


[[<Figure size 3840x1955 with 5 Axes>]]